In [115]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os
from tkinter import ttk

ApprovedName = ['jh', 'junhong']
df = None
df2 = None
merged_df = None
columns = []


def set_window_geometry(window, width, height):
    """Set the geometry of the window to be centered on the screen."""
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    center_x = int((screen_width / 2) - (width / 2))
    center_y = int((screen_height / 2) - (height / 2))
    window.geometry(f"{width}x{height}+{center_x}+{center_y}")


def submit_form(event=None):
    """Handles form submission and transitions to a new window."""
    name = entry.get()
    print(f"User: {name}")
    root.destroy()
    open_new_window(name)


def open_file(current_window, target="df"):
    """Handles file upload and saves to the specified DataFrame."""
    global df, df2
    file_path = filedialog.askopenfilename(
        title="Select a File",
        filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")]
    )
    if file_path:
        file_extension = file_path.split(".")[-1].lower()
        try:
            data = pd.read_csv(file_path) if file_extension == "csv" else pd.read_excel(file_path)
            if target == "df":
                df = data
            elif target == "df2":
                df2 = data
            messagebox.showinfo("Success", f"File loaded: {os.path.basename(file_path)}")
            current_window.destroy()
            if target == "df":
                choose_function_page()
            elif target == "df2":
                data_merge_interface()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file:\n{e}")

def function_selection_handler(new_window):
    """Handle the function based on the user's selection."""
    global columns
    function_type = function_type_selection.get().strip()
    if function_type == "merge":
        uploading_file_2()
    elif function_type == "remove null from a specific column":
        removing_null_specific_column()
    elif function_type == "Select specific columns":
        new_window.destroy()
        columns = list(df.columns)
        create_column_selection_window(columns)
    else:
        print("Invalid function selected.")
        messagebox.showerror("Error", "Invalid function selected.")
    # Close the current window after action
    if new_window.winfo_exists():
        new_window.destroy()

def column_selection_handler(new_window):
    """remove the nulls from the selected column"""
    global df_dropped_null_from_column
    df_column_selection = df_columns_selection.get()
    
    try:
        # Ensure the window hasn't been destroyed manually
        if new_window.winfo_exists():
            new_window.destroy()
        df_dropped_null_from_column = df.dropna(subset=[df_column_selection])
        export_window(df_dropped_null_from_column)
    except Exception as e:
        print(f"An error occurred: {e}")
        messagebox.showerror("Error", f"An error occurred: {e}")

def submit_column_name(event=None):
    """Submit the column name and merge type to merge the data."""
    global merged_df
    col_name = column_selection.get().strip()  # Get the selected column from dropdown
    merge_type = merge_type_selection.get().strip()  # Get the selected merge type from dropdown

    # Validate the selected column and merge type
    if col_name in df.columns and col_name in df2.columns:
        if merge_type in ["left", "right", "inner", "outer"]:
            try:
                # Perform the merge
                merged_df = pd.merge(df, df2, on=col_name, how=merge_type)
                messagebox.showinfo("Success", "Merge successful!")
                new_window.destroy()  # Close the input window
                export_window(merged_df)  # Proceed to export
            except Exception as e:
                messagebox.showerror("Error", f"An error occurred during merge: {e}")
        else:
            # Invalid merge type error
            messagebox.showerror("Error", "Invalid merge type selected. Please try again.")
    else:
        # Column not found error
        messagebox.showerror("Error", f"Column '{col_name}' not found in both data tables.")

def export_window(dataframe):
    """Create a window to allow exporting the merged DataFrame."""
    export_win = tk.Tk()
    export_win.title("Export Data")
    set_window_geometry(export_win, 300, 200)

    tk.Label(export_win, text="Click button to export file").pack(pady=10)
    
    def save_file():
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv"), ("All files", "*.*")])
        if file_path:
            dataframe.to_csv(file_path, index=False)
            messagebox.showinfo("Export", "File saved successfully!")
            export_win.destroy()

    tk.Button(export_win, text="Save to CSV", command=save_file).pack(pady=20)
    export_win.mainloop()

    
    
####################################################
    

def create_form_window():
    """Creates and displays the initial form window."""
    global root, entry
    root = tk.Tk()
    root.title("Login")
    set_window_geometry(root, 300, 200)

    tk.Label(root, text="Enter your name:").pack(pady=10)
    entry = tk.Entry(root)
    entry.pack(pady=10)
    entry.focus_force()
    tk.Button(root, text="Submit", command=submit_form).pack(pady=10)
    root.bind('<Return>', lambda event: submit_form())

    # Bring the window to the forefront
    root.lift()  # Raise the window above others
    root.attributes('-topmost', True)  # Keep it temporarily on top
    root.after(100, lambda: root.attributes('-topmost', False))  # Allow other windows on top afterward
    
    root.mainloop()


def open_new_window(name):
    """Open a new window for authorized users."""
    new_window = tk.Tk()
    new_window.title("Welcome Page - Upload first base file")
    set_window_geometry(new_window, 300, 200)
    if name in ApprovedName:
        tk.Label(new_window, text=f"Welcome, {name}! \n Let's upload the first base file.").pack(pady=50)
        tk.Button(new_window, text="Upload File", command=lambda: open_file(new_window, target="df")).pack(pady=20)
    else:
        tk.Label(new_window, text=f"Sorry {name}, You are unauthorized.").pack(pady=50)
        tk.Button(new_window, text="Close", command=new_window.destroy).pack(pady=10)
    new_window.mainloop()

def choose_function_page():
    """allow user to choose which function they want to do with the uploaded first file"""
    global function_type_selection
    new_window = tk.Tk()
    new_window.title("Data Function Page")
    set_window_geometry(new_window, 300, 200)
    
    # Create a dropdown menu for function options
    function_types = ["merge", "remove null from a specific column", "Select specific columns"]
    function_type_selection = ttk.Combobox(new_window, values=function_types, state="readonly", width=40)
    function_type_selection.pack(pady=20)
    
    # Button to confirm function selection
    tk.Button(new_window, text="Submit", command=lambda: function_selection_handler(new_window)).pack(pady=10)
    
    # Button to close the window
    tk.Button(new_window, text="Close", command=new_window.destroy).pack(pady=10)
    
    new_window.mainloop()
        
    
def uploading_file_2():
    """Upload second file df2."""
    new_window = tk.Tk()
    new_window.title("Upload second file")
    set_window_geometry(new_window, 300, 200)

    tk.Label(new_window, text="Upload the second file.").pack(pady=50)
    tk.Button(new_window, text="Upload File", command=lambda: open_file(new_window, target="df2")).pack(pady=20)
    new_window.mainloop()


def data_merge_interface():
    """Options to manipulate data."""
    global new_window, column_selection, merge_type_selection
    new_window = tk.Tk()
    new_window.title("Data Merge Interface")
    set_window_geometry(new_window, 400, 400)

    tk.Label(new_window, text="Select the column to merge on:").pack(pady=10)

    # Create a dropdown menu for columns
    combined_columns = list(set(df.columns).intersection(set(df2.columns)))
    column_selection = ttk.Combobox(new_window, values=combined_columns, state="readonly", width=40)
    column_selection.pack(pady=5)
    column_selection.focus()

    tk.Label(new_window, text="Merge type:").pack(pady=10)

    # Create a dropdown menu for merge type
    merge_types = ["left", "right", "inner", "outer"]
    merge_type_selection = ttk.Combobox(new_window, values=merge_types, state="readonly", width=40)
    merge_type_selection.pack(pady=5)

    tk.Button(new_window, text="Submit", command=submit_column_name).pack(pady=10)
    new_window.bind('<Return>', submit_column_name)

    # Add a frame for scrollable text
    frame = tk.Frame(new_window)
    frame.pack(pady=10, fill=tk.BOTH, expand=True)

    # Add a scrollbar to the frame
    scrollbar = tk.Scrollbar(frame)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Add a Text widget with word wrapping
    text_widget = tk.Text(frame, wrap=tk.WORD, yscrollcommand=scrollbar.set, height=8, width=50)
    text_widget.pack(fill=tk.BOTH, expand=True)
    scrollbar.config(command=text_widget.yview)

    # Display column labels from both dataframes
    text_widget.insert(tk.END, f"1st table columns:\n{', '.join(list(df.columns))}\n\n")
    text_widget.insert(tk.END, f"2nd table columns:\n{', '.join(list(df2.columns))}")

    # Disable editing in the Text widget
    text_widget.config(state=tk.DISABLED)

    new_window.mainloop()


def removing_null_specific_column():
    """Removing null from specific column"""
    global df_columns_selection
    new_window = tk.Tk()
    new_window.title("Removing null in a Column")
    set_window_geometry(new_window, 300, 200)
    
    # Create a dropdown menu for merge type
    df_columns = list(df.columns)
    df_columns_selection = ttk.Combobox(new_window, values=df_columns, state="readonly", width=40)
    df_columns_selection.pack(pady=5)
    df_columns_selection.focus()

    # Button to confirm column selection
    tk.Button(new_window, text="Submit", command=lambda: column_selection_handler(new_window)).pack(pady=10)
    
    new_window.mainloop()
    

def create_column_selection_window(columns):
    """Create a column selection window with search, select all, and deselect all functionality."""

    # Create the main window
    window = tk.Tk()
    window.title("Select Columns")

    # Get the screen dimensions for a fullscreen-like experience
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    window.geometry(f"{screen_width}x{screen_height}+0+0")

    # Create a search bar
    search_label = tk.Label(window, text="Search Columns:")
    search_label.pack(pady=5)

    search_entry = tk.Entry(window, width=40)
    search_entry.pack(pady=5)

    # Create a canvas to hold checkboxes and add a scrollbar
    canvas = tk.Canvas(window)
    canvas.pack(side="left", fill="both", expand=True)

    # Vertical scrollbar for the canvas
    v_scrollbar = tk.Scrollbar(window, orient="vertical", command=canvas.yview)
    v_scrollbar.pack(side="right", fill="y")

    canvas.configure(yscrollcommand=v_scrollbar.set)

    # Create a frame inside the canvas for checkboxes
    checkboxes_frame = tk.Frame(canvas)
    canvas.create_window((0, 0), window=checkboxes_frame, anchor="nw")

    # Dictionary to store checkboxes and their associated BooleanVars
    checkboxes = {}

    def filter_columns(*args):
        """Filter checkboxes based on the search term."""
        search_term = search_entry.get().lower()
        for col, info in checkboxes.items():
            checkbox = info['widget']
            if search_term in col.lower():
                checkbox.grid(row=info['row'], column=info['col'], sticky='w')
            else:
                checkbox.grid_forget()

    # Bind the search entry field to dynamically filter columns
    search_entry.bind("<KeyRelease>", filter_columns)

    # Create checkboxes for each column
    row, col = 0, 0
    max_rows = 10  # Max rows before moving to the next column
    for idx, column in enumerate(columns):
        check_var = tk.BooleanVar()
        checkbox = tk.Checkbutton(checkboxes_frame, text=column, variable=check_var)
        checkbox.grid(row=row, column=col, sticky='w', padx=5, pady=5)

        # Store checkbox info for filtering
        checkboxes[column] = {'widget': checkbox, 'var': check_var, 'row': row, 'col': col}

        # Adjust row/column placement
        row += 1
        if row >= max_rows:  # Start a new column
            row = 0
            col += 1

    # Update the scrollable region
    checkboxes_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

    # Select/Deselect All Buttons
    def select_all():
        for checkbox in checkboxes.values():
            checkbox['var'].set(True)

    def deselect_all():
        for checkbox in checkboxes.values():
            checkbox['var'].set(False)

    select_all_button = tk.Button(window, text="Select All", command=select_all, width=20)
    select_all_button.pack(pady=10)

    deselect_all_button = tk.Button(window, text="Deselect All", command=deselect_all, width=20)
    deselect_all_button.pack(pady=10)

    # Submit Button
    def submit():
        selected_columns = [col for col, checkbox in checkboxes.items() if checkbox['var'].get()]
        #print("Selected Columns:", selected_columns)
        dataframe = df[selected_columns]
        export_window(dataframe)
        window.destroy()

    submit_button = tk.Button(window, text="Submit", command=submit, width=20)
    submit_button.pack(pady=20)
    
    tk.Button(window, text="Close", command=window.destroy).pack(pady=10)
    submit_button.pack(pady=5)
    # Start the Tkinter event loop
    window.mainloop()

    
# Run the form creation function
create_form_window()


User: jh
Selected Columns: ['age', 'no_minimum_protection']
Selected Columns: ['age', 'no_minimum_protection', 'minimum_protection']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\temas\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\temas\AppData\Local\Temp\ipykernel_23596\304669061.py", line 361, in submit
    window.destroy()
  File "C:\Users\temas\anaconda3\Lib\tkinter\__init__.py", line 2368, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\temas\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\temas\AppData\Local\Temp\ipykernel_23596\304669061.py", line 361, in submit
    window.destroy()
  File "C:\Users\temas\anaconda3\Lib\tkinter\__init__.py", line 2368, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" comman

In [ ]:
current_dir =os.getcwd()
parent_dir = os.path.dirname(current_dir)
parent_dir